## Charts Setup
*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

In [ ]:
! pip install --upgrade altair >> pip.log

Import chart packages and define chart functions:

In [ ]:
import altair as alt


def plot_release_count(release_counts):
    # check if input contains the right columns
    if not set(["collection_id", "release_type", "release_count", "ocid_count"]).issubset(release_counts.columns):
        raise ValueError(
            "Input data must contain the following columns: collection_id, release_type, release_count, ocid_count"
        )
    chart = (
        alt.Chart(release_counts)
        .mark_bar()
        .encode(
            x=alt.X(
                "release_count",
                type="ordinal",
                axis=alt.Axis(title="release count", labelAngle=0),
            ),
            y=alt.Y(
                "ocid_count",
                type="quantitative",
                axis=alt.Axis(title="ocid count", format="~s", tickCount=5),
            ),
            color=alt.Color(
                "release_type",
                type="nominal",
                title="release type",
                scale=alt.Scale(range=["#D6E100", "#FB6045", "#23B2A7", "#6C75E1"]),
            ),
            tooltip=[
                alt.Tooltip("release_count", title="release count"),
                alt.Tooltip("ocid_count", title="ocid count", format="~s"),
                alt.Tooltip("release_type", title="release type"),
                alt.Tooltip("collection_id", title="collection id"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )

    return chart


def plot_objects_per_stage(objects_per_stage):
    # check if input contains the right columns
    if not set(["stage", "object_count"]).issubset(objects_per_stage.columns):
        raise ValueError("Data must contain columns 'stage' and 'object_count'")
    # draw chart
    chart = (
        alt.Chart(objects_per_stage)
        .mark_bar(fill="#d6e100")
        .encode(
            x=alt.X(
                "stage",
                type="ordinal",
                scale=alt.Scale(
                    domain=[
                        "planning",
                        "tender",
                        "awards",
                        "contracts",
                        "implementation",
                    ]
                ),
                sort=["planning", "tender", "awards", "contracts", "implementation"],
                axis=alt.Axis(title="stage", labelAngle=0),
            ),
            y=alt.Y(
                "object_count",
                type="quantitative",
                axis=alt.Axis(title="number of objects", format="~s", tickCount=5),
            ),
            tooltip=[
                alt.Tooltip("stage", title="stage"),
                alt.Tooltip("object_count", title="number of objects"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )
    return chart


def plot_releases_by_month(release_dates):
    # check if input contains the right columns
    if not set(["date", "collection_id", "release_type", "release_count"]).issubset(release_dates.columns):
        raise ValueError("Data must contain columns 'date', 'collection_id', 'release_type', 'release_count'")
    # check if number of rows is more than 5000
    if release_dates.shape[0] > 5000:
        alt.data_transformers.disable_max_rows()

    # draw chart
    chart = (
        alt.Chart(release_dates)
        .mark_line(strokeWidth=3)
        .encode(
            x=alt.X("date", timeUnit="yearmonth", axis=alt.Axis(title="year and month")),
            y=alt.Y(
                "release_count",
                type="quantitative",
                aggregate="sum",
                axis=alt.Axis(title="number of releases", format="~s", tickCount=5),
                scale=alt.Scale(zero=False),
            ),
            color=alt.Color(
                "release_type",
                type="nominal",
                scale=alt.Scale(range=["#D6E100", "#FB6045", "#23B2A7", "#6C75E1"]),
                legend=alt.Legend(title="release type"),
            ),
            tooltip=[
                alt.Tooltip("date", timeUnit="yearmonth", title="date"),
                alt.Tooltip("release_count", aggregate="sum", title="number of releases"),
                alt.Tooltip("release_type", title="release type"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )
    return chart


def plot_objects_per_year(objects_per_year):
    # check if input contains the right columns
    if not set(["year", "tenders", "awards"]).issubset(objects_per_year.columns):
        raise ValueError("Data must contain columns 'year', 'tenders' and 'awards'")
    # draw chart
    chart = (
        alt.Chart(objects_per_year)
        .transform_fold(["tenders", "awards"])
        .mark_line(strokeWidth=3)
        .encode(
            x=alt.X(
                "year",
                type="quantitative",
                axis=alt.Axis(title="year", format=".0f", tickCount=dates.shape[0]),
            ),
            y=alt.Y(
                "value",
                type="quantitative",
                axis=alt.Axis(title="number of objects", format="~s", tickCount=5),
                scale=alt.Scale(zero=False),
            ),
            color=alt.Color(
                "key",
                type="nominal",
                title="object type",
                scale=alt.Scale(domain=["tenders", "awards"], range=["#D6E100", "#FB6045"]),
            ),
            tooltip=[
                alt.Tooltip("year", title="year", type="quantitative"),
                alt.Tooltip("value", title="number of objects", type="quantitative"),
                alt.Tooltip("key", title="object type", type="nominal"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )
    return chart


def plot_top_buyers(buyers):
    # check if input contains the right columns
    if not set(["name", "total_tenders"]).issubset(buyers.columns):
        raise ValueError("Data must contain columns 'name' and 'total_tenders'")
    # draw chart
    chart = (
        alt.Chart(buyers)
        .mark_bar(fill="#d6e100")
        .encode(
            x=alt.X(
                "total_tenders",
                type="quantitative",
                axis=alt.Axis(title="number of tenders", format="~s", tickCount=5),
            ),
            y=alt.Y(
                "name",
                type="ordinal",
                axis=alt.Axis(title="buyer", labelAngle=0),
                sort=alt.SortField("total_tenders", order="descending"),
            ),
            tooltip=[
                alt.Tooltip("name", title="buyer", type="nominal"),
                alt.Tooltip("total_tenders", title="number of tenders", type="quantitative"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )
    return chart